In [ ]:
import pandas as pd
import xlwings as xw
import matplotlib.pyplot as plt
from datetime import datetime
from os

EXPORT_PATH = '/full/path/to/export/dir/'

In [ ]:
# Excel File Version
excel_file = 'path/to/file.xlsx'
wb = xw.Book(excel_file)
sheet = wb.sheets[0]
# first end last element in sheet -> maybe just for password protected excel files
df = sheet['A1:AX15366'].options(pd.DataFrame, index=False, header=True).value

# CSV File Version, with seperation and encoding
#df = pd.read_csv('path/to/data.csv', sep='\t', encoding='utf-8')

#show dataframe
df

In [ ]:
# there are many entries without page action definition or with lazy-load and request
# these entries may not direct user interactions, so we remove them for further analysis

clean_df = df[ df['page_action'].notnull() & (df['page_action'] != 'lazy-load') & (df['page_action'] != 'request')]
clean_df

In [ ]:
# calculate the click count per day on amazon
df_hitdate = pd.DataFrame({'count' : clean_df.groupby(['hit_day']).size()}).reset_index()
df_hitdate.to_csv(os.path.join(EXPORT_PATH, 'hit_day_count_alldata.csv'),sep='\t', encoding='utf-8')

In [ ]:
# the following cells are for orders in your data
# since amazon uses different keywords for orders, you need to check them manually
# save them in a file and have a look
pd.value_counts(clean_df['page_action']).to_csv(os.path.join(EXPORT_PATH, 'check_page_actions_for_orders.csv'),sep='\t', encoding='utf-8')

In [ ]:
# the following query on the data searches for the identified keywords
# saves the data for a visualization

mb = clean_df.loc[ 
    (clean_df['page_action'] == 'PlaceOrder-1') |
    (clean_df['page_action'] == 'PlaceOrder-2') |
    (clean_df['page_action'] == 'PlaceOrder-3') |
    (clean_df['page_action'] == 'PlaceOrder-5') |
    (clean_df['page_action'] == 'PlaceOrder-6') |
    (clean_df['page_action'] == 'PlaceOrder-11') |
    (clean_df['page_action'] == 'PlaceOrder-15') |
    (clean_df['page_action'] == 'Purchase') ]
maybe_orders = mb[['hit_datetime', 'hit_day']]
maybe_orders['order'] = True
maybe_orders.to_csv(os.path.join(EXPORT_PATH, 'maybe_orders.csv'), sep='\t', encoding='utf-8')

In [ ]:
# concat activity with orders
activity_orders = pd.merge(df_hitdate, maybe_orders, on='hit_day', how='left')
del activity_orders['hit_datetime']
activity_orders.to_csv(os.path.join(EXPORT_PATH, 'activity_and_orders.csv'), sep='\t', encoding='utf-8')

In [ ]:
# how many books did you order?
# check the data for any 'addtocart' keyword and check if the previous tabname was books (this seems to be a book you added to your cart)
books = clean_df.loc[ ((clean_df['page_action'] == 'AddToCart') & (clean_df['from_tab_name'] == 'books')) | ((clean_df['page_action'] == 'AddToCard') & (clean_df['from_tab_name'] == 'books-intl-de'))  ]

books.loc[:, 'year'] = books['hit_day'].dt.year
books.loc[:, 'month'] = books['hit_day'].dt.month

book_count = pd.DataFrame({'count' : books.groupby(['year', 'month']).size()}).reset_index()
book_count

book_count.to_csv(os.path.join(EXPORT_PATH, 'ordered_books.csv'), sep='\t', encoding='utf-8')